<a href="https://colab.research.google.com/github/abhicse002/nlp_projects/blob/main/Implementing_a_Retrieval_Augmented_Generation_(RAG)_System_with_OpenAI's%C2%A0API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
!pip install langchain
!pip install openai
!pip install pypdf
!pip install pytesseract pdf2image
!pip install chromadb
!pip install langchain-openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [51]:
# f = open('/content/ts_openapi_key.txt')
# api_key = f.read()
# os.environ['OPENAI_API_KEY'] = api_key
# openai.api_key= os.getenv('OPENAI_API_KEY')

In [76]:
import pytesseract
from pdf2image import convert_from_path
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document  # Import the Document class
import os

In [77]:
def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)  # Convert PDF pages to images
    extracted_documents = []
    # Performing OCR on each image
    for i, image in enumerate(images):
        text = pytesseract.image_to_string(image)
        doc = Document(page_content=text, metadata={"page": i + 1}) # Create a Document instance for each page
        extracted_documents.append(doc)

    return extracted_documents  # Return a list of Document instances, we cant pass text string.

pdf_path = "/content/Priynashu.pdf"
text = extract_text_from_pdf(pdf_path)
print(text)


[Document(metadata={'page': 1}, page_content=' \n\n \n\nWent? MAHARASHTRA 0 2022 0 BN 582257\n\na,\n\n \n\nThis Outbound Transport Agreement (“Agreement”) is made at Mumbai on this 7"\nday of June, 2022;\n\nBETWEEN\n\nADITYA BIRLA FASHION AND RETAIL LIMITED, a company duly\nincorporated under the Companies Act, 1956 and having its registered office at Piramal\nAgastya Corporate Park, Building \'A’, 4 and 5" Floor, Unit No. 401, 403, 501, 502,\nL.B.S. Road, Kurla, Mumbai - 400 070, through Pantaloons Division (hereinafter\nreferred to as “ABFRL”, which expression shall, unless it be repugnant to the meaning\n\nor context thereof, mean and include its successors and permitted assigns) of the FIRST\nPART;\n\n88\nj 7 0}\n<2 |B peLH! }S)\n= em —}\nS\n\nCw\nY x\n\x0c'), Document(metadata={'page': 2}, page_content=' \n\nWENT, MAHARASHTRA @ 2022 0 O7AA 65432\n\nEL =\n\n \n\n \n \n\nVella agi raters, Aas\n\nyafim LO00008\n21 APR 2022\nara sitet\n\n    \n \n  \n\n  \n\nAND\n\nPRIYANSHU VENTURES 

In [74]:
# Step 3: Create Embeddings and Vector Store
embeddings = OpenAIEmbeddings()
try:
    vector_store = FAISS.from_documents(text, embeddings)
except IndexError:
    print("Error: Embeddings list appears to be empty. Check API key and setup.")
    raise

# Proceed with setting up the QA chain and asking questions
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

# Example questions
questions = [
    "What is the Diesel price to be considered?",
     "What is the rate between Jamalpur DC to MEERUT?"
]

# Generate answers
for question in questions:
    answer = qa_chain.run(question)
    print(f"Q: {question}\nA: {answer}\n")


Q: What is the Diesel price to be considered?
A:  The Diesel price to be considered is Rs. 97.09/- per liter, as stated in the context as the base for calculation.

Q: What is the rate between Jamalpur DC to MEERUT?
A:  The rate between Jamalpur DC to MEERUT is Rs. 15000/- per trip for a BOLERO vehicle, with an additional fixed rate of Rs. 500 per trip per store for a total of 06 stores. 

